In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Hyper-parameters
image_size = 64
num_epochs = 200
batch_size = 100
nz = 100 # size of latent vector z
ngf = 64 # size of hidden layer in Generator
ndf = 64 # size of hidden layer in Discrimitor
niter = 25 # number of epoch for training
lr = 0.0002 # learning rate
nc = 1 # number of channel. In MNIST, it is grayscale.
ngpu = 1 # number of GPU
beta1 = 0.5 # beta1 for Adam
root_dir = '/content/drive/MyDrive/MLDL/DCGAN'
sample_dir = root_dir + '/samples_DCGAN'

cuda


In [ ]:
# Change current path
os.chdir(root_dir)
print(os.getcwd())

/content/drive/MyDrive/MLDL/DCGAN


In [ ]:
transform=transforms.Compose([
                              transforms.Resize(image_size),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                           ])

In [ ]:
# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias)

In [ ]:
class Generator(nn.Module):
  def __init__(self, ngpu):
    super(Generator, self).__init__()
    self.main = nn.Sequential(
        # input is Z, going into a convolution
        nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
        nn.BatchNorm2d(ngf * 8),
        nn.ReLU(True),
        # state size. (ngf*8) x 4 x 4
        nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
        nn.BatchNorm2d(ngf * 4),
        nn.ReLU(True),
        # state size. (ngf*4) x 8 x 8
        nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
        nn.BatchNorm2d(ngf * 2),
        nn.ReLU(True),
        # state size. (ngf*2) x 16 x 16
        nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
        nn.BatchNorm2d(ngf),
        nn.ReLU(True),
        # state size. (ngf) x 32 x 32
        nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
        nn.Tanh()
        # state size. (nc) x 64 x 64
    )

  def forward(self, input):
    output = self.main(input)
    return output

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1).squeeze(1)

In [ ]:
netG = Generator(ngpu).to(device)
netG.apply(weights_init)
print(netG)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [ ]:
netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
print(netD)

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [ ]:
from torchsummary import summary as summary_
summary_(netD,(1,64,64),batch_size=100)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [100, 64, 32, 32]           1,024
         LeakyReLU-2          [100, 64, 32, 32]               0
            Conv2d-3         [100, 128, 16, 16]         131,072
       BatchNorm2d-4         [100, 128, 16, 16]             256
         LeakyReLU-5         [100, 128, 16, 16]               0
            Conv2d-6           [100, 256, 8, 8]         524,288
       BatchNorm2d-7           [100, 256, 8, 8]             512
         LeakyReLU-8           [100, 256, 8, 8]               0
            Conv2d-9           [100, 512, 4, 4]       2,097,152
      BatchNorm2d-10           [100, 512, 4, 4]           1,024
        LeakyReLU-11           [100, 512, 4, 4]               0
           Conv2d-12             [100, 1, 1, 1]           8,192
          Sigmoid-13             [100, 1, 1, 1]               0
Total params: 2,763,520
Trainable param

In [ ]:
summary_(netG,(100,1,1),batch_size=100)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1           [100, 512, 4, 4]         819,200
       BatchNorm2d-2           [100, 512, 4, 4]           1,024
              ReLU-3           [100, 512, 4, 4]               0
   ConvTranspose2d-4           [100, 256, 8, 8]       2,097,152
       BatchNorm2d-5           [100, 256, 8, 8]             512
              ReLU-6           [100, 256, 8, 8]               0
   ConvTranspose2d-7         [100, 128, 16, 16]         524,288
       BatchNorm2d-8         [100, 128, 16, 16]             256
              ReLU-9         [100, 128, 16, 16]               0
  ConvTranspose2d-10          [100, 64, 32, 32]         131,072
      BatchNorm2d-11          [100, 64, 32, 32]             128
             ReLU-12          [100, 64, 32, 32]               0
  ConvTranspose2d-13           [100, 1, 64, 64]           1,024
             Tanh-14           [100, 1,

In [ ]:
criterion = nn.BCELoss()

fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

In [ ]:
# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
for epoch in range(niter):
    for i, data in enumerate(data_loader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].to(device)
        label = torch.full((batch_size,), real_label,
                           dtype=real_cpu.dtype, device=device)
        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, niter, i, len(data_loader),
                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % sample_dir,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),
                    '%s/fake_samples_epoch_%03d.png' % (sample_dir, epoch),
                    normalize=True)
    # do checkpointing
    #torch.save(netG.state_dict(), '%s/DCnetG_epoch_%d.pth' % (root_dir, epoch))
    #torch.save(netD.state_dict(), '%s/DCnetD_epoch_%d.pth' % (root_dir, epoch))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[16/25][400/600] Loss_D: 0.0175 Loss_G: 5.6660 D(x): 0.9935 D(G(z)): 0.0107 / 0.0061
[16/25][401/600] Loss_D: 0.0304 Loss_G: 5.2999 D(x): 0.9898 D(G(z)): 0.0193 / 0.0079
[16/25][402/600] Loss_D: 0.0454 Loss_G: 4.5624 D(x): 0.9750 D(G(z)): 0.0190 / 0.0147
[16/25][403/600] Loss_D: 0.0418 Loss_G: 4.0521 D(x): 0.9732 D(G(z)): 0.0138 / 0.0269
[16/25][404/600] Loss_D: 0.0424 Loss_G: 3.9822 D(x): 0.9749 D(G(z)): 0.0163 / 0.0299
[16/25][405/600] Loss_D: 0.0223 Loss_G: 4.8234 D(x): 0.9917 D(G(z)): 0.0137 / 0.0132
[16/25][406/600] Loss_D: 0.0350 Loss_G: 5.3589 D(x): 0.9931 D(G(z)): 0.0273 / 0.0071
[16/25][407/600] Loss_D: 0.0451 Loss_G: 5.9654 D(x): 0.9923 D(G(z)): 0.0358 / 0.0034
[16/25][408/600] Loss_D: 0.1448 Loss_G: 1.3345 D(x): 0.8777 D(G(z)): 0.0026 / 0.3402
[16/25][409/600] Loss_D: 1.2369 Loss_G: 14.8936 D(x): 0.9992 D(G(z)): 0.6290 / 0.0000
[16/25][410/600] Loss_D: 10.0290 Loss_G: 5.3934 D(x): 0.0001 D(G(z)): 0.0000 / 0.0095
[16/25][411/600] Loss_D: 2.